# Постановка задачи.

Обучите классификатор, предсказывающий категорию объявления на Авито по его заголовку, описанию и цене. Метрика для оценки качества -- accuracy. Необходимо предоставить прокомментированный код (желательно на Python 2.x или 3.x, можно в Jupiter Notebook) для всех этапов решения задачи и результат скоринга файла test.csv с помощью предложенного классификатора (csv-файл с двумя столбцами: item_id, category_id).

Категории имеют иерархическую структуру, описанную в файле сategory.csv. Посчитайте также accuracy вашей модели на каждом уровне иерархии.

Для решения задачи можно использовать любые внешние модели, но не внешние данные.


# Подключение необходимых библиотек

In [1]:
# Импорт библиотеки, которая понадобится для наглядного представления csv-файлов в виде таблиц,
# получения сводных данных по таблице
import pandas as pd
# Импорт библиотеки, которая понадобится для разбиения выборки на обучающую и тестовую,
# кросс-валидации и поиска оптимальных гиперпараметров модели по сетке
import sklearn.model_selection as ms
# Импорт библиотек для работы с текстовыми признаками
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Импорт используемого в задаче классификатора
from sklearn.naive_bayes import MultinomialNB 
# Импорт библиотеки, для упрощения поиска оптимальных гиперпараметров модели по сетке
from sklearn.pipeline import Pipeline
# Импорт библиотеки, содержащей метрику accuracy
from sklearn import metrics
# Импорт библиотеки, содержащей стоп-слова для русского языка
from nltk.corpus import stopwords

# Загрузка данных

Загрузка csv-файла _train.csv_ с обучающей выборкой в объект DataFrame. 
-  index_col - название столбца в файле, значения которого будут использоваться как индексы строк
- encoding - указание кодировки исходного файла

In [2]:
ads = pd.read_csv('train.csv', index_col='item_id', encoding = 'utf-8')

Посмотрим первые 10 записей в таблице.

In [3]:
ads.head(10)

,title,description,price,category_id
item_id,,,,
0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1
5,Бас гитара invasion bg110,Состояние хорошее. Имеется теплый чехол .,3999.0,50
6,"Смесь ""Грудничок"" г. Зеленодольск",Смесь молочная адаптированная ультрапастеризов...,15.0,41
7,G-shock,Часы абсолютно новые! с коробкой. Часы Китай...,2500.0,36
8,"Санатории Белоруссии. - ""Лепельский военный""",Санатории Белоруссии! - «Лепельский военный» ...,1090.0,48


Видно, что в в талице имеется 4 столбца:
    - title - заголовок объявления
    - description - описание объявления
    - price - указанная цена
    - category_id - номер категории, которому принадлежит объвление

_title_ и _description_  - признаки в виде текстов. Для того, чтобы их можно было использовать в обучении их нужно преобразовать в удобный для обучения формат.

Проверим, есть ли незаполненные ячейки в таблице.

In [4]:
ads.isnull().any()

title          False
description    False
price          False
category_id    False
dtype: bool

Все ячейки заполнены. Узнаем колличество строк.

In [5]:
ads.shape

(489517, 4)

Итого 489 517 строк. Определим баланс классов в обучающей выборке.

In [6]:
class_frequency = pd.crosstab(index=ads.category_id, columns="count")

In [7]:
class_frequency

col_0,count
category_id,
0,8862
1,8022
2,9887
3,8604
4,8616
5,8241
6,8697
7,8592
8,8698


Видно, что в обучающей выборке объявлений разных категорий примерно одинаковое колличество. Число различных меток классов: __54__.

Отделим данные, которые будем использовать для обучения (data) от меток классов (target).

In [8]:
target = ads.category_id
data = ads[['title','description','price']]

# Предобработка данных и обучение
В качестве семейства алгоритмов для обучения будем использовать семейство __Наивных байесовских классификаторов__. В этом случае, для обучения будем использовать только текстовые признаки.

## Предобработка данных

Создадим на основе предложенной ниже функции _preprocessingNB(...)_, из старой таблицы (data) новую (dataNB), которая будет содержать только один столбец, представляющий собой заголовок объявления и его описание.

In [9]:
def preprocessingNB(data):
    return pd.DataFrame({'full_descr': data.title +' '+data.description})

Воспользуемся этой функцей.

In [10]:
dataNB = preprocessingNB(data)

Посмотрим первые 10 элементов новой таблицы.

In [11]:
dataNB.head(10)

,full_descr
item_id,
0,Картина Гобелен. Размеры 139х84см.
1,Стулья из прессованной кожи Продам недорого 4 ...
2,"Домашняя мини баня Мини баня МБ-1(мини сауна),..."
3,"Эксклюзивная коллекция книг ""Трансаэро"" + пода..."
4,Ноутбук aser Продаётся ноутбук ACER e5-511C2TA...
5,Бас гитара invasion bg110 Состояние хорошее. И...
6,"Смесь ""Грудничок"" г. Зеленодольск Смесь молочн..."
7,G-shock Часы абсолютно новые! с коробкой. Часы...
8,"Санатории Белоруссии. - ""Лепельский военный"" С..."


Для оценки качества полученных алгоритмов и оптимизации гиперпараметров модели разделим выборку на:
- обучающую (X_train, y_train)
- тестовую (X_test, y_test).

Размер тестовой выборки *test_size* = 0.3

Выбран параметр *sratify* = target, для того, чтобы баланс классов в тестовой и обучающей выборке был одинаков. Это необходимо для корректного обучения (избегается ситуация, когда в обучающую выборку не попали какие-то классы) и сравнения алгоритмов.

In [12]:
X_train, X_test, y_train, y_test = ms.train_test_split(dataNB, target, test_size = 0.3, stratify = target,
                                                       random_state = 42)

Был выбран параметр random_state = 42, чтобы train_test_split каждый раз выдавал фиксированное разбиение.

## Обучение с параметрами по умолчанию

Для предварительной оценки качества модели, используем все используемые ниже функции с параметрами по умолчанию.

Перед тем как обучать модель, преобразуем столбец *full_descr* в удобный для обучения формат. Для этого воспользуемся функцией *CountVectorizer()* - для русского языка она поддерживает только токенизацию текста. Текст разбивается на слова, на основе слов создается словарь, число слов в словаре - новое число признаков. Значение каждого нового признака - число вхождений данного слова в данной записи.

In [14]:
%%time
# Создается объект класса CountVectorizer
count_vect = CountVectorizer()
# Токенизация и создание словаря на основе X_train.full_descr.tolist(), где
# X_train.full_descr.tolist() - выдает список значений столбца full_descr таблицы X_train
count_vect.fit(X_train.full_descr.tolist())

Wall time: 16.5 s


*%%time* - показывает время выполнения 

Вычислим число слов в словаре.

In [15]:
len(count_vect.vocabulary_)

419766

Таким образом, 1 текстовый признак *full_descr* будет преобразован в 420 442 числовых признаков.
Причем число новых признаков такого же порядка величины, как и число записей (489 517).
Уменьшение колличества неинформативных признаков должно увеличить качество модели.

С помощью метода transform() объекта count_vect, преобразуем тестовую и обучающую выборку.

In [16]:
%%time
X_train_counts = count_vect.transform(X_train.full_descr.tolist())
X_test_counts = count_vect.transform(X_test.full_descr.tolist())

Wall time: 18.8 s


С помощью атрибута shape объекта X_train_counts можно убедиться в том, что число новых признаков изменилось и равно числу элементов в словаре.

In [17]:
X_train_counts.shape

(342661, 419766)

Обучим Наивный байесовский классификатор для случая многоклассовой классификации.

In [24]:
%%time
# Создается объект класса MultinomialNB - Наивный байесовский классификатор для случая многоклассовой классификации
clf = MultinomialNB()
# Обучаем этот алгоритм на обучающей выборке
clf.fit(X_train_counts, y_train)
# Получаем предсказания алгоритма на тестовой выборке
predicted = clf.predict(X_test_counts)
# Сравниваем предсказанные значения с истинными значениями меток тестовой выборки
# с помощью метрики accuracy_score
acc = metrics.accuracy_score(y_test, predicted)
# Выводим значение точности
print(acc)

0.855756659585
Wall time: 3.32 s


Как видно, Наивный байесовский классификатор для случая многоклассовой классификации дал верные предсказания в 85% случаев.
Но использовались гиперпараметры по умолчанию. Попробуем улучшить модель подбирая гиперпараметры.

## Поиск оптимальных гиперпараметров

CountVectorizer() - имеет такой параметр как stop_words. Стоп-слова - это общие слова документов, которые не являются специфичными или разделяющими для разных классов. Библиотека *nltk* содержит характерные стоп-слова для всего русского языка в целом. Используя параметр stop_words мы исключим их из словаря.

In [19]:
stop_words = stopwords.words('russian')

Для поиска оптимальных параметров создим функцию *check_vect_MultiNB(parameters_grid)*,
которая на вход будет принимать параметры для поиска по сетке, выводить на экран отчет по поиску оптимальных параметров.

In [20]:
def check_vect_MultiNB(parameters_grid):
    # Создадим объект Pipeline - "конвеерный" объект, который по очереди будет действовать
    # на входящий объект: сначала его векторизует методами класса CountVectorizer, 
    # потом обучит на нем объект класса MultinomialNB. На выходе получим обученный
    # объект класса MultinomialNB.
    # Используем стоп-слова при создании словаря.
    pipe_counts_MultiNB = Pipeline(steps=[('vect', CountVectorizer(stop_words=stop_words)),
                                          ('clf', MultinomialNB())])
    
    # Для поиска оптимальных гиперпараметров по сетке, будем использовать кросс-валидацию на 2 блока,
    # причем с учетом баланса классов (чтобы в каждом блоке баланс классов был такой же, как и в
    # исходной обучающей выборке).
    cv = ms.StratifiedShuffleSplit(n_splits=2)
    # Создадим объект для поиска по сетке, передадим в него:
    # "конвеерный" объект, параметры сетки, используемую метрику для сравнения (accuracy),
    # число потоков выполнения и индексы для кросс-валидации.
    grid_cv = ms.GridSearchCV(pipe_counts_MultiNB, parameters_grid, scoring = 'accuracy', n_jobs=5, cv=cv)
    
    # Обучим объект для поиска по сетке на обучающей выборке
    grid_cv.fit(X_train.full_descr.tolist(), y_train)
    # Получим предсказания наилучшего алгоритма (с наивысшей точностью) на тестовой выборке
    test_predictions = grid_cv.best_estimator_.predict(X_test.full_descr.tolist())
    # Получим точность наилучшего алгоритма
    acc = metrics.accuracy_score(y_test, test_predictions)
    # Выведем на экран точность
    print('accuracy =', acc)
    # Наилучшие параметры алгоритма
    print('best :', grid_cv.best_params_)
    # Средние значения точности по кросс-валидации на два блока для каждого значения из parameters_grid
    print('Отчет:')
    for t in zip(grid_cv.cv_results_['params'], grid_cv.cv_results_['mean_test_score']):
        print(t)
        
    # Возвратить обученный объект класса GridSearchCV
    return grid_cv

### min_df
Для начала настроим параметр *min_df* из *CountVectorizer()*. Когда строится словарь, то игнорируются слова, которые встречаются в документе строго меньшее *min_df* число раз.

In [21]:
%%time
# Зададим значения min_df для которых будем считать точность
vect__min_df = [1, 2, 3, 4, 5]
# Создадим parameters_grid соответсвующий значениям min_df
parameters_grid = {'vect__min_df' : vect__min_df}
# Найдем оптимальные параметры
ans = check_vect_MultiNB(parameters_grid)

accuracy = 0.860475567903
best : {'vect__min_df': 2}
Отчет:
({'vect__min_df': 1}, 0.85893133335278837)
({'vect__min_df': 2}, 0.85939825488078914)
({'vect__min_df': 3}, 0.85891674205503843)
({'vect__min_df': 4}, 0.85779321212828674)
({'vect__min_df': 5}, 0.85710742113403571)
Wall time: 2min 28s


Таким образом, **min_df = 2** - некоторое оптимальное значение. Удалось немного увеличить качество.

Определим размер словаря для такого параметра.

In [22]:
# Создается и обучается словарь
c = CountVectorizer(stop_words=stop_words, min_df=2).fit(X_train.full_descr.tolist(), y_train).vocabulary_
# Выводится значение объема словаря на экран
print(len(c))

193626


Размер словаря сократился практически в 2 раза 

### max_features

Настроим параметр *max_features* из *CountVectorizer()*. Словарь ограничен числом наиболее употребляемых слов равным *max_features*.

Согласно:
http://www.myvocab.info/articles/slovarniy-zapas-nositeley-russkogo-yazyka-vliyanie-vozrasta-i-obrazovaniya
число наиболее употребляемых слов находится в интервале от 30000 до 100000

In [25]:
%%time
# Зададим значения max_features для которых будем считать точность
vect__max_features = [30000, 60000, 90000, 120000, 150000, 180000, 210000]
# Создадим parameters_grid соответсвующий найденному значению min_df, и
# значениям max_features
parameters_grid = {'vect__min_df' : [2],
                   'vect__max_features' : vect__max_features}
# Найдем оптимальные параметры
ans = check_vect_MultiNB(parameters_grid)

accuracy = 0.860475567903
best : {'vect__max_features': 210000, 'vect__min_df': 2}
Отчет:
({'vect__max_features': 30000, 'vect__min_df': 2}, 0.84563866110251851)
({'vect__max_features': 60000, 'vect__min_df': 2}, 0.85491872647153233)
({'vect__max_features': 90000, 'vect__min_df': 2}, 0.85776402953278663)
({'vect__max_features': 120000, 'vect__min_df': 2}, 0.85898969854378848)
({'vect__max_features': 150000, 'vect__min_df': 2}, 0.85982140251553973)
({'vect__max_features': 180000, 'vect__min_df': 2}, 0.85998190679079001)
({'vect__max_features': 210000, 'vect__min_df': 2}, 0.86002568068404006)
Wall time: 3min 2s


**Оптимально выбирать словарь с наибольшим числом слов.**

### max_df

Настроим параметр *max_df* из *CountVectorizer()*. Когда строится словарь, то игнорируются слова, которые встречаются в документе с частотой строго большей, чем *max_df*.

In [26]:
%%time
# Зададим значения max_df для которых будем считать точность
vect__max_df = [1.0, 0.8, 0.6, 0.5, 0.1]
# Создадим parameters_grid соответсвующий найденному значению min_df, и
# значениям max_df.
parameters_grid = {'vect__min_df': [2],
                   'vect__max_df' : vect__max_df}
# Найдем оптимальные параметры
ans = check_vect_MultiNB(parameters_grid)

accuracy = 0.862368578744
best : {'vect__max_df': 0.1, 'vect__min_df': 2}
Отчет:
({'vect__max_df': 1.0, 'vect__min_df': 2}, 0.86004027198179001)
({'vect__max_df': 0.8, 'vect__min_df': 2}, 0.86004027198179001)
({'vect__max_df': 0.6, 'vect__min_df': 2}, 0.86004027198179001)
({'vect__max_df': 0.5, 'vect__min_df': 2}, 0.86004027198179001)
({'vect__max_df': 0.1, 'vect__min_df': 2}, 0.86234569702629349)
Wall time: 2min 25s


Определим размер словаря для такого параметра

In [27]:
# Создается и обучается словарь
c = CountVectorizer(stop_words=stop_words,
                    min_df=2,
                    max_df = 0.1).fit(X_train.full_descr.tolist(), y_train).vocabulary_
# Выводится значение объема словаря на экран
print(len(c))

193622


Попробуем взять *max_df* еще меньше.

In [29]:
%%time
# Зададим значения max_df для которых будем считать точность
vect__max_df = [0.1, 0.09, 0.08, 0.05, 0.01]
# Создадим parameters_grid соответсвующий найденному значению min_df, и
# значениям max_df.
parameters_grid = {'vect__min_df': [2],
                   'vect__max_df' : vect__max_df}
# Найдем оптимальные параметры
ans = check_vect_MultiNB(parameters_grid)

accuracy = 0.864404586806
best : {'vect__max_df': 0.05, 'vect__min_df': 2}
Отчет:
({'vect__max_df': 0.1, 'vect__min_df': 2}, 0.85942743747628914)
({'vect__max_df': 0.09, 'vect__min_df': 2}, 0.85948580266728924)
({'vect__max_df': 0.08, 'vect__min_df': 2}, 0.85964630694253952)
({'vect__max_df': 0.05, 'vect__min_df': 2}, 0.8607114716782911)
({'vect__max_df': 0.01, 'vect__min_df': 2}, 0.84086730673826127)
Wall time: 2min 27s


Попробуем взять *max_df* еще меньше.

In [30]:
%%time
# Зададим значения max_df для которых будем считать точность
vect__max_df = [0.05, 0.04, 0.03]
# Создадим parameters_grid соответсвующий найденному значению min_df, и
# значениям max_df.
parameters_grid = {'vect__min_df': [2],
                   'vect__max_df' : vect__max_df}
# Найдем оптимальные параметры
ans = check_vect_MultiNB(parameters_grid)

accuracy = 0.864404586806
best : {'vect__max_df': 0.05, 'vect__min_df': 2}
Отчет:
({'vect__max_df': 0.05, 'vect__min_df': 2}, 0.86438847871129654)
({'vect__max_df': 0.04, 'vect__min_df': 2}, 0.86341086176204507)
({'vect__max_df': 0.03, 'vect__min_df': 2}, 0.86306067061604463)
Wall time: 2min


Определим размер словаря для такого параметра

In [31]:
# Создается и обучается словарь
c = CountVectorizer(stop_words=stop_words,
                    min_df=2,
                    max_df = 0.05).fit(X_train.full_descr.tolist(), y_train).vocabulary_
# Выводится значение объема словаря на экран
print(len(c))

193604


Таким образом, **max_df = 0.05** - некоторое оптимальное значение. Удалось немного увеличить качество.

### TF-IDF

Попробуем увеличить качество модели TF-IDF подход:
- учтем словоупотребление каждого слова в каждой записи - скомпенсирует колличество словоупотреблений в длинных и коротких записях
- снизим вес низкоинформативных слов (которые очень часто употребляются в записях независимо от категории)

In [36]:
def check_TfidfVec_MultiNB(parameters_grid):
    # Создадим объект Pipeline - "конвеерный" объект, который по очереди будет действовать
    # на входящий объект: сначала его векторизует методами класса TfidfVectorizer
    # с учетом словоупотребления и информативности, 
    # потом обучит на нем объект класса MultinomialNB. На выходе получим обученный
    # объект класса MultinomialNB.
    # Используем стоп-слова при создании словаря и найденные оптимальные значения
    # min_df=2, max_df=0.5.
    pipe_counts_MultiNB = Pipeline(steps=[('TfidfVec', TfidfVectorizer(stop_words=stop_words,
                                                                   min_df=2,
                                                                   max_df=0.5)),
                                          ('clf', MultinomialNB())])

    # Для поиска оптимальных гиперпараметров по сетке, будем использовать кросс-валидацию на 2 блока,
    # причем с учетом баланса классов (чтобы в каждом блоке баланс классов был такой же, как и в
    # исходной обучающей выборке).
    cv = ms.StratifiedShuffleSplit(n_splits=2)
    # Создадим объект для поиска по сетке, передадим в него:
    # "конвеерный" объект, параметры сетки, используемую метрику для сравнения (accuracy),
    # число потоков выполнения и индексы для кросс-валидации.
    grid_cv = ms.GridSearchCV(pipe_counts_MultiNB, parameters_grid, scoring = 'accuracy', n_jobs=5, cv=cv)
    
    # Обучим объект для поиска по сетке на обучающей выборке
    grid_cv.fit(X_train.full_descr.tolist(), y_train)
    # Получим предсказания наилучшего алгоритма (с наивысшей точностью) на тестовой выборке
    test_predictions = grid_cv.best_estimator_.predict(X_test.full_descr.tolist())
    # Получим точность наилучшего алгоритма
    acc = metrics.accuracy_score(y_test, test_predictions)
    # Выведем на экран точность
    print('accuracy =', acc)
    # Наилучшие параметры алгоритма
    print('best :', grid_cv.best_params_)
    # Средние значения точности по кросс-валидации на два блока для каждого значения из parameters_grid
    print('Отчет:')
    for t in zip(grid_cv.cv_results_['params'], grid_cv.cv_results_['mean_test_score']):
        print(t)
        
    # Возвратить обученный объект класса GridSearchCV
    return grid_cv

### Определим оптимальные параметры, отвечающие за TF-IDF

In [37]:
%%time
# Создадим parameters_grid соответсвующий всевозможным значениям параметров,
# отвечающих за TF-IDF.
parameters_grid = {'TfidfVec__sublinear_tf': [True, False],
                   'TfidfVec__norm' : ['l1', 'l2', None],
                   'TfidfVec__use_idf' : [True, False]}
# Найдем оптимальные параметры
ans = check_TfidfVec_MultiNB(parameters_grid)

accuracy = 0.863519365909
best : {'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True}
Отчет:
({'TfidfVec__norm': 'l1', 'TfidfVec__sublinear_tf': True, 'TfidfVec__use_idf': True}, 0.83692765634575539)
({'TfidfVec__norm': 'l1', 'TfidfVec__sublinear_tf': True, 'TfidfVec__use_idf': False}, 0.8083578953512125)
({'TfidfVec__norm': 'l1', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True}, 0.84047334169901067)
({'TfidfVec__norm': 'l1', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': False}, 0.81235591093471848)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': True, 'TfidfVec__use_idf': True}, 0.86287098374529425)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': True, 'TfidfVec__use_idf': False}, 0.84512796568126769)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True}, 0.86397992237429599)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': False}, 0.8468789214112703)
({'TfidfVec__

Таким образом, **sublinear_tf = False,
               norm = 'l2',
               use_idf = True** - некоторые оптимальные значения. Удалось немного увеличить качество

### alpha

Настроим параметр *alpha* из *MultinomialNB()*. Параметр сглаживания *alpha* необходим для того, чтобы слова, которые не встречались в обучающей выборке, но есть в тестовой, не давали нулевые вероятности при вычислениях.

In [38]:
%%time
# Зададим значения alpha для которых будем считать точность
alpha = [0.8, 0.6, 0.5, 0.4, 0.3]
# Создадим parameters_grid соответсвующий найденным значению sublinear_tf, norm, и use_idf
# значениям alpha.
parameters_grid = {'TfidfVec__sublinear_tf': [False],
                   'TfidfVec__norm' : ['l2'],
                   'TfidfVec__use_idf' : [True],
                   'clf__alpha': alpha}
# Найдем оптимальные параметры
ans = check_TfidfVec_MultiNB(parameters_grid)

accuracy = 0.870151713243
best : {'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.3}
Отчет:
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.8}, 0.86469489596404703)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.6}, 0.86662094726704997)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.5}, 0.86752560772755127)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.4}, 0.86860536376105291)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.3}, 0.869305746053054)
Wall time: 2min 33s


Попробуем взять *alpha* еще меньше.

In [39]:
%%time
# Зададим значения alpha для которых будем считать точность
alpha = [0.35, 0.3, 0.25, 0.2, 0.1]
# Создадим parameters_grid соответсвующий найденным значению sublinear_tf, norm, и use_idf
# значениям alpha.
parameters_grid = {'TfidfVec__sublinear_tf': [False],
                   'TfidfVec__norm' : ['l2'],
                   'TfidfVec__use_idf' : [True],
                   'clf__alpha': alpha}
# Найдем оптимальные параметры
ans = check_TfidfVec_MultiNB(parameters_grid)

accuracy = 0.871186740753
best : {'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.2}
Отчет:
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.35}, 0.87000612834505497)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.3}, 0.87056059765955585)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.25}, 0.87114424956955672)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.2}, 0.87134852773805704)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.1}, 0.87129016254705693)
Wall time: 2min 31s


Попробуем взять *alpha* еще меньше.

In [40]:
%%time
# Зададим значения alpha для которых будем считать точность
alpha = [0.09, 0.08, 0.07, 0.06, 0.05]
# Создадим parameters_grid соответсвующий найденным значению sublinear_tf, norm, и use_idf
# значениям alpha.
parameters_grid = {'TfidfVec__sublinear_tf': [False],
                   'TfidfVec__norm' : ['l2'],
                   'TfidfVec__use_idf' : [True],
                   'clf__alpha': alpha}
# Найдем оптимальные параметры
ans = check_TfidfVec_MultiNB(parameters_grid)

accuracy = 0.871956201994
best : {'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.08}
Отчет:
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.09}, 0.87064814544605595)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.08}, 0.87072110193480612)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.07}, 0.87069191933930601)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.06}, 0.87035631949105552)
({'TfidfVec__norm': 'l2', 'TfidfVec__sublinear_tf': False, 'TfidfVec__use_idf': True, 'clf__alpha': 0.05}, 0.86993317185630492)
Wall time: 2min 28s


Таким образом, **alpha = 0.08** - некоторое оптимальное значение. Удалось немного увеличить качество.

#### Таким образом, лучшие параметры:
min_df : 2

max_df : 0.05

sublinear_tf : False

norm : 'l2'

use_idf : True

alpha : 0.08

## Лучший классификатор

Таким образом, выделим лучший классификатор:

In [42]:
best_alg = ans.best_estimator_

In [104]:
# Получим еще раз предсказания на тесте от лучшего классификатора
test_predictions = best_alg.predict(X_test.full_descr.tolist())
# Получим точность наилучшего алгоритма
acc = metrics.accuracy_score(y_test, test_predictions)
# Выведем на экран точность такого классификатора
print(acc)

0.871956201994


Таким образом, точность лучшего классификатора равна : **0.871956201994**

# Обработка файла с категориями

Загрузка csv-файла _category.csv_ с описанием категорий в объект DataFrame. 
-  index_col - название столбца в файле, значения которого будут использоваться как индексы строк
- encoding - указание кодировки исходного файла

In [106]:
cats = pd.read_csv('category.csv', index_col='category_id', encoding= 'utf-8')

Выведем таблицу с категориями на экран

In [107]:
cats

,name
category_id,
0,Бытовая электроника|Телефоны|iPhone
1,Бытовая электроника|Ноутбуки
2,Бытовая электроника|Телефоны|Samsung
3,Бытовая электроника|Планшеты и электронные кни...
4,"Бытовая электроника|Игры, приставки и программ..."
5,Бытовая электроника|Аудио и видео|Телевизоры и...
6,Бытовая электроника|Телефоны|Другие марки
7,Бытовая электроника|Настольные компьютеры
8,"Бытовая электроника|Игры, приставки и программ..."


Для удобства преобразуем таблицу в словарь (с помощью генератора словарей), где ключом будет служить название категории, а значением - его *category_id*.

In [108]:
category_dict = {v[1]: v[0] for v in zip(range(0, len(cats)), cats.name.tolist())}

Посмотрим этот словарь

In [51]:
category_dict

{'Бытовая электроника|Аудио и видео|Акустика, колонки, сабвуферы': 12,
 'Бытовая электроника|Аудио и видео|Телевизоры и проекторы': 5,
 'Бытовая электроника|Игры, приставки и программы|Игровые приставки': 4,
 'Бытовая электроника|Игры, приставки и программы|Игры для приставок': 8,
 'Бытовая электроника|Настольные компьютеры': 7,
 'Бытовая электроника|Ноутбуки': 1,
 'Бытовая электроника|Планшеты и электронные книги|Планшеты': 3,
 'Бытовая электроника|Телефоны|Nokia': 10,
 'Бытовая электроника|Телефоны|Samsung': 2,
 'Бытовая электроника|Телефоны|Sony': 11,
 'Бытовая электроника|Телефоны|iPhone': 0,
 'Бытовая электроника|Телефоны|Аксессуары|Чехлы и плёнки': 9,
 'Бытовая электроника|Телефоны|Другие марки': 6,
 'Бытовая электроника|Товары для компьютера|Мониторы': 13,
 'Бытовая электроника|Товары для компьютера|Сетевое оборудование': 14,
 'Для дома и дачи|Бытовая техника|Для дома|Стиральные машины': 26,
 'Для дома и дачи|Бытовая техника|Для кухни|Мелкая кухонная техника': 29,
 'Для дома и д

Найдем максимальную глубину иерархии категорий. Видно, что один уровень иерархии разделяется от другого символом **|**

In [109]:
# Инициализация максимальной глубины нулем
max_len=0

# Создание цикла по элементам ключей словаря category_dict
for key in category_dict.keys():
    
    # Каждый ключ представляет собой строку, эту строку разбиваем на
    # массив строк с помощью split('|').
    splited_key = key.split('|')
    
    # Число элементов массива равно глубине иерархии, поэтому если 
    # число элементов массива больше max_len, вычисленной на прошлой итерации,
    # то max_len = len(splited_key)
    if len(splited_key) > max_len:
        max_len = len(splited_key)

# Вывод максимальной глубины 
print('max_len =',max_len)

max_len = 4


Получим все категории на уровне 0. Результат получим в виде словаря, где ключ - название категории, а значение - список состоящий из тех category_id, которые входят в данную категорию.

In [55]:
# Инициализация словаря на уровне 0 пустым словарем.
set0={}

# Создание цикла по элементам ключей словаря category_dict
for key in category_dict.keys():
    
    # Каждый ключ представляет собой строку, эту строку разбиваем на
    # массив строк с помощью split('|').
    splited_key = key.split('|')
    
    # splited_key[0] - название ключа на глубине 0
    # если в словаре set0 этот ключ встречается в первый раз, то 
    # он добавляется в словарь (создается список из одного элемента: category_dict[key])
    # если не в первый раз, то к его значению добавляется category_dict[key].
    # Такое разделение необходимо, чтобы создать список.
    if splited_key[0] in set0.keys():
        set0[splited_key[0]].append(category_dict[key])  # ключ уже есть в словаре
    else:
        set0[splited_key[0]] = [category_dict[key]]      # ключ встречается в первый раз
        
# Вывод словаря        
print(set0)

{'Бытовая электроника': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'Для дома и дачи': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'Личные вещи': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'Хобби и отдых': [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]}


Получим все категории на уровне 1. Результат получим в виде словаря, где ключ - название категории, а значение - список состоящий из тех category_id, которые входят в данную категорию.

In [112]:
# Инициализация словаря на уровне 1 пустым словарем.
set1={}

# Создание цикла по элементам ключей словаря category_dict
for key in category_dict.keys():
    
    # Каждый ключ представляет собой строку, эту строку разбиваем на
    # массив строк с помощью split('|').
    splited_key = key.split('|')
    
    # Проверяем, чтобы существовал элемент splited_key[1].
    # Если это так, то продолжаем создание словаря.
    # В качестве названия ключа берется '|'.join(splited_key[0:2])
    if len(splited_key) >= 2:
        
        if '|'.join(splited_key[0:2]) in set1.keys():
            set1['|'.join(splited_key[0:2])].append(category_dict[key])
        else:
            set1['|'.join(splited_key[0:2])]= [category_dict[key]]
            
# Вывод словаря
print(set1)

{'Бытовая электроника|Телефоны': [0, 2, 6, 9, 10, 11], 'Бытовая электроника|Ноутбуки': [1], 'Бытовая электроника|Планшеты и электронные книги': [3], 'Бытовая электроника|Игры, приставки и программы': [4, 8], 'Бытовая электроника|Аудио и видео': [5, 12], 'Бытовая электроника|Настольные компьютеры': [7], 'Бытовая электроника|Товары для компьютера': [13, 14], 'Для дома и дачи|Ремонт и строительство': [15, 17, 21, 25], 'Для дома и дачи|Мебель и интерьер': [16, 18, 19, 22, 23, 27, 28], 'Для дома и дачи|Посуда и товары для кухни': [20], 'Для дома и дачи|Растения': [24], 'Для дома и дачи|Бытовая техника': [26, 29], 'Личные вещи|Товары для детей и игрушки': [30, 32, 34, 41], 'Личные вещи|Одежда, обувь, аксессуары': [31, 33, 35, 38, 39], 'Личные вещи|Часы и украшения': [36, 40], 'Личные вещи|Красота и здоровье': [37], 'Хобби и отдых|Спорт и отдых': [42, 46, 47], 'Хобби и отдых|Книги и журналы': [43, 51], 'Хобби и отдых|Коллекционирование': [44, 45], 'Хобби и отдых|Билеты и путешествия': [48], '

Далее аналогично. 

In [113]:
set2={}

for key in category_dict.keys():

    splited_key = key.split('|')

    if len(splited_key) >= 3:
        if '|'.join(splited_key[0:3]) in set2.keys():
            set2['|'.join(splited_key[0:3])].append(category_dict[key])
        else:
            set2['|'.join(splited_key[0:3])]= [category_dict[key]]
            
    
print(set2)

{'Бытовая электроника|Телефоны|iPhone': [0], 'Бытовая электроника|Телефоны|Samsung': [2], 'Бытовая электроника|Планшеты и электронные книги|Планшеты': [3], 'Бытовая электроника|Игры, приставки и программы|Игровые приставки': [4], 'Бытовая электроника|Аудио и видео|Телевизоры и проекторы': [5], 'Бытовая электроника|Телефоны|Другие марки': [6], 'Бытовая электроника|Игры, приставки и программы|Игры для приставок': [8], 'Бытовая электроника|Телефоны|Аксессуары': [9], 'Бытовая электроника|Телефоны|Nokia': [10], 'Бытовая электроника|Телефоны|Sony': [11], 'Бытовая электроника|Аудио и видео|Акустика, колонки, сабвуферы': [12], 'Бытовая электроника|Товары для компьютера|Мониторы': [13], 'Бытовая электроника|Товары для компьютера|Сетевое оборудование': [14], 'Для дома и дачи|Ремонт и строительство|Стройматериалы': [15], 'Для дома и дачи|Мебель и интерьер|Кровати, диваны и кресла': [16], 'Для дома и дачи|Ремонт и строительство|Инструменты': [17], 'Для дома и дачи|Мебель и интерьер|Шкафы и комоды'

In [114]:
set3={}

for key in category_dict.keys():

    splited_key = key.split('|')

    if len(splited_key) >= 4:
        if '|'.join(splited_key[0:4]) in set3.keys():
            set3['|'.join(splited_key[0:4])].append(category_dict[key])
        else:
            set3['|'.join(splited_key[0:4])]= [category_dict[key]]
            
            
print(set3)

{'Бытовая электроника|Телефоны|Аксессуары|Чехлы и плёнки': [9], 'Для дома и дачи|Бытовая техника|Для дома|Стиральные машины': [26], 'Для дома и дачи|Бытовая техника|Для кухни|Мелкая кухонная техника': [29], 'Личные вещи|Одежда, обувь, аксессуары|Женская одежда|Верхняя одежда': [33], 'Личные вещи|Одежда, обувь, аксессуары|Женская одежда|Платья и юбки': [35], 'Личные вещи|Одежда, обувь, аксессуары|Женская одежда|Другое': [38], 'Личные вещи|Одежда, обувь, аксессуары|Женская одежда|Обувь': [39]}


Можно было это все загнать в один цикл, но так кажется нагляднее.

Объеденим все эти словари в один

In [59]:
# Инициализация пустым словарем
all_categories_dict = {}

# Добавления словаря один за другим
all_categories_dict.update(set0)
all_categories_dict.update(set1)
all_categories_dict.update(set2)
all_categories_dict.update(set3)

Выведем целиком словарь

In [62]:
all_categories_dict

{'Бытовая электроника': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'Бытовая электроника|Аудио и видео': [5, 12],
 'Бытовая электроника|Аудио и видео|Акустика, колонки, сабвуферы': [12],
 'Бытовая электроника|Аудио и видео|Телевизоры и проекторы': [5],
 'Бытовая электроника|Игры, приставки и программы': [4, 8],
 'Бытовая электроника|Игры, приставки и программы|Игровые приставки': [4],
 'Бытовая электроника|Игры, приставки и программы|Игры для приставок': [8],
 'Бытовая электроника|Настольные компьютеры': [7],
 'Бытовая электроника|Ноутбуки': [1],
 'Бытовая электроника|Планшеты и электронные книги': [3],
 'Бытовая электроника|Планшеты и электронные книги|Планшеты': [3],
 'Бытовая электроника|Телефоны': [0, 2, 6, 9, 10, 11],
 'Бытовая электроника|Телефоны|Nokia': [10],
 'Бытовая электроника|Телефоны|Samsung': [2],
 'Бытовая электроника|Телефоны|Sony': [11],
 'Бытовая электроника|Телефоны|iPhone': [0],
 'Бытовая электроника|Телефоны|Аксессуары': [9],
 'Бытовая электроника|Телефон

# Точность лучшего классификатора для каждой из категорий

Вспомогательная функция, которая преобразует метки классов: если они принадлежат некоторому множеству, то заменяются на 1. Если не принадлежат, то на 0.

In [115]:
def to_0_1(test_predictions, values):
    ans = []
    for element in test_predictions:
        if element in values:
            ans.append(1)
        else:
            ans.append(0)
    return ans

Функция которая выводит точность каждой категории с учетом иерархии категорий. Ответ выдается в виде словаря, где ключ - название категории, а значение - точность для нее.

In [116]:
def accuracy_for_all_cat(y_test, test_predictions):
    ans = {}
    for key in all_categories_dict.keys():
        values = all_categories_dict[key]
        acc = metrics.accuracy_score(to_0_1(y_test, values), to_0_1(test_predictions, values))
        ans.update({key : acc})
    return ans

Воспользуемся этой функцией и найдем точность для каждой из категорий с учетом иерархии

In [117]:
my_ans = accuracy_for_all_cat(y_test, test_predictions)

Отобразим результат

In [118]:
my_ans

{'Бытовая электроника': 0.99338127144958321,
 'Бытовая электроника|Аудио и видео': 0.99637059432369124,
 'Бытовая электроника|Аудио и видео|Акустика, колонки, сабвуферы': 0.9972353870458136,
 'Бытовая электроника|Аудио и видео|Телевизоры и проекторы': 0.99876750013618787,
 'Бытовая электроника|Игры, приставки и программы': 0.99864493108895791,
 'Бытовая электроника|Игры, приставки и программы|Игровые приставки': 0.99667020755025326,
 'Бытовая электроника|Игры, приставки и программы|Игры для приставок': 0.99688129868714936,
 'Бытовая электроника|Настольные компьютеры': 0.99834531786239578,
 'Бытовая электроника|Ноутбуки': 0.99806613281037204,
 'Бытовая электроника|Планшеты и электронные книги': 0.99680639538050875,
 'Бытовая электроника|Планшеты и электронные книги|Планшеты': 0.99680639538050875,
 'Бытовая электроника|Телефоны': 0.99374216919976033,
 'Бытовая электроника|Телефоны|Nokia': 0.99609140927166751,
 'Бытовая электроника|Телефоны|Samsung': 0.99396006972816908,
 'Бытовая электро

Максимальная и минимальная точность

In [139]:
print('Минимальная точность: ', min(my_ans.values()))
print('Максимальная точность: ', max(my_ans.values()))

Минимальная точность:  0.973777033284
Максимальная точность:  0.999591436509


# Предсказание категорий для файла test.csv

Загрузим файл *test.csv*, для которого нужно сдлеать предсказания в таблицу

In [119]:
data_test = pd.read_csv('test.csv', index_col='item_id', encoding = 'utf-8')

Отобразим первые 5 строк

In [120]:
data_test.head()

,title,description,price
item_id,,,
489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0


Преобразуем его к формату, который принимает лучший классификатор

In [121]:
dataNB_test = preprocessingNB(data_test)

Отобразим первые 5 строк

In [122]:
dataNB_test.head()

,full_descr
item_id,
489517,Стоик журнальный сталь продам журнальный столи...
489518,iPhone 5 64Gb Телефон в хорошем состоянии. Ком...
489519,Утеплитель ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина у...
489520,Пальто демисезонное Продам пальто женское (бук...
489521,"Samsung syncmaster T200N Условно рабочий, проб..."


Сделаем предсказания лучшим классификатором

In [123]:
predictions = best_alg.predict(dataNB_test.full_descr.tolist())

Представим результат предсказания в виду таблицы.

In [128]:
answer = pd.DataFrame({'category_id' : predictions}, index = dataNB_test.index)

Отобразим первые 5 строк

In [134]:
answer.head()

,category_id
item_id,
489517,22
489518,6
489519,15
489520,33
489521,5


Запишем ответ в csv-файл

In [136]:
answer.to_csv('answer.csv')

_Иванов Кирилл Сергеевич, 15.10.2017_